In [1]:
# !pip install beautifulsoup4
# !pip install html5lib
# !pip install python-whois

     ---------------------------------------- 109.6/109.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103247 sha256=59110ddc00b832772224f37e27662b0ccd0fcef8a4a14fa9405704444f77b59f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\e6\e9\d3\1e41a6c95b398de12c5a332ff28805aa44e68aa317ea60266d
Successfully built python-whois


In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

import whois

In [3]:
# gets html of webpage
url = 'https://www.secureworks.com/blog/opsec-mistakes-reveal-cobalt-mirage-threat-actors'
result = requests.get(url)
soup = BeautifulSoup(result.text, "html.parser")

#extracts IP addresses, URLs and hashes from table
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
display(df)

,Indicator,Type,Context
0,gupdate.us,Domain name,TunnelFish C2 server used by COBALT MIRAGE
1,msupdate.top,Domain name,TunnelFish C2 server used by COBALT MIRAGE
2,193.142.59.174,IP address,Hosting TunnelFish domains used by COBALT MIRAGE
3,172.245.26.118,IP address,Staging and distributing COBALT MIRAGE malware
4,mssync.one,Domain name,Suspected C2 server linked to COBALT MIRAGE
5,upmirror.top,Domain name,Suspected C2 server linked to COBALT MIRAGE
6,104.168.117.149,IP address,Hosting COBALT MIRAGE domains
7,69314c1969f28bfab34683769286326e25d9a0f07 c4b...,SHA256 hash,TunnelFish malware used by COBALT MIRAGE
8,f38f3a1cda90229434e8ab8c59342838106b9778,SHA1 hash,TunnelFish malware used by COBALT MIRAGE
9,00e4c488558492b80fd27d51b159a099,MD5 hash,TunnelFish malware used by COBALT MIRAGE


In [5]:
#removes all whitespaces and display results in a list
extracted = [x.replace(" ", "") for x in df['Indicator']]
print(extracted)

['gupdate.us', 'msupdate.top', '193.142.59.174', '172.245.26.118', 'mssync.one', 'upmirror.top', '104.168.117.149', '69314c1969f28bfab34683769286326e25d9a0f07c4bad3443d08efe4f43e0a8', 'f38f3a1cda90229434e8ab8c59342838106b9778', '00e4c488558492b80fd27d51b159a099']


In [68]:
#gets domain and ip addresses 
df_domains = df.loc[df['Type'].isin(['Domain name', 'IP address'])]
domains = df_domains['Indicator']
# print(domains)

#empty df to store results
results_df = pd.DataFrame()

for domain in domains:
    #extracting whois information for each domain
    whois_info = whois.whois(domain)
    whois_info_df = pd.DataFrame(whois_info.items())
    
    #transpose df for csv format
    whois_info_df = whois_info_df.T
    #set header and reset index
    whois_info_df = whois_info_df.rename(columns=whois_info_df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
    
    #add domain extracted as new column
    whois_info_df.insert(0, 'Domain/IP', domain)
    
    #append results
    results_df = pd.concat([results_df, whois_info_df], ignore_index=True)

#export to csv, fill NA values
display(results_df)
results_df.to_csv("taskA.csv", encoding='utf-8', index=False, na_rep='NA')

0         gupdate.us
1       msupdate.top
2     193.142.59.174
3     172.245.26.118
4         mssync.one
5       upmirror.top
6    104.168.117.149
Name: Indicator, dtype: object


,Domain/IP,domain_name,domain__id,whois_server,registrar,registrar_id,registrar_url,registrar_email,registrar_phone,status,...,updated_date,referral_url,emails,dnssec,name,org,address,city,state,country
0,gupdate.us,gupdate.us,DD745E60FBAD445BBA4F822873796C6A3-GDREG,whois.namecheap.com,NAMECHEAP INC,1068,http://www.namecheap.com,abuse@namecheap.com,"[+1.6613102107, +1.9854014545]",[clientTransferProhibited https://icann.org/ep...,...,"[2023-06-06 09:22:11, 0001-01-01 00:00:00]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,msupdate.top,None,NaN,None,None,NaN,NaN,NaN,NaN,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,193.142.59.174,None,NaN,None,None,NaN,NaN,NaN,NaN,ASSIGNED PA,...,None,NaN,"[abuse@ripe.net, hostmaster@ripe.net, 'abuse@h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,172.245.26.118,"[COLOCROSSING.COM, colocrossing.com]",NaN,whois.names4ever.com,"Deluxe Small Business Sales, Inc. d/b/a Aplus.net",NaN,NaN,NaN,NaN,[clientTransferProhibited https://icann.org/ep...,...,2023-06-22 15:16:46,NaN,"[dns@cs.aplus.net, dns@aplus.net, corp-domains...","[unsigned, Unsigned]",Domain Admin,"Deluxe Small Business Sales, Inc. Aplus.net",801 Marquette Ave S 801 Marquette Ave S,Minneapolois,MN,US
4,mssync.one,mssync.one,NaN,https://porkbun.com/whois,Porkbun,NaN,NaN,NaN,NaN,[pendingDelete https://icann.org/epp#pendingDe...,...,2023-08-09 12:04:40,NaN,abuse@porkbun.com,unsigned,REDACTED FOR PRIVACY,"Private by Design, LLC",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,NC,US
5,upmirror.top,upmirror.top,NaN,whois.namesilo.com,"NameSilo,LLC",NaN,NaN,NaN,NaN,[clientTransferProhibited https://icann.org/ep...,...,2023-08-02 17:33:53,NaN,abuse@namesilo.com,unsigned,REDACTED FOR PRIVACY,See PrivacyGuardian.org,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,AZ,US
6,104.168.117.149,"[COLOCROSSING.COM, colocrossing.com]",NaN,whois.names4ever.com,"Deluxe Small Business Sales, Inc. d/b/a Aplus.net",NaN,NaN,NaN,NaN,[clientTransferProhibited https://icann.org/ep...,...,2023-06-22 15:16:46,None,"[dns@cs.aplus.net, dns@aplus.net, corp-domains...","[unsigned, Unsigned]",Domain Admin,"Deluxe Small Business Sales, Inc. Aplus.net",801 Marquette Ave S 801 Marquette Ave S,Minneapolois,MN,US


In [59]:
# import session_info
# session_info.show() python-whois-0.8.0